<a href="https://colab.research.google.com/github/AnnaFattakhova/Dataset_literature_curriculum/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D0%B8_%D0%B2_%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Замены вручную (для merged1)
1.  ‘first publication’ => ‘first_publication’
2.  для базы 1 изменено значение uid (теперь начинается не с 1_0, а с 1_1)
3. заменены NA:
  *  в  century, type, oblig_author,  oblig_title, section, number_author, number_title, source_issue, title_cycle, group, notes на 'inapp'
  *  в author на 'Неизвестный автор'
  *  в profound на '0'





# Добавлены вручную (при желании можно автоматизировать)

1. добавлен перед “notes” столбец “translated”
2. добавлен перед “notes” столбец “sort”

# Добавлены автоматически

Для столбца translated

In [ ]:
#в type для значений "зарубежная": В столбец translated вставить ‘1’.
df.loc[df['type'] == 'зарубежная', 'translated'] = 1

Для столбца source_year

In [ ]:
#заменяем пустые значения в "source_year" на значение из "source_id"
df['source_year'] = df['source_year'].fillna(df['source_id'])

# Удалены (автоматически)

1.   столбец first_publication
2.   столбец quantity


In [ ]:
#убираем столбец first_publication
df = df.drop('first_publication', axis = 1)

#убираем столбец quantity
df = df.drop('quantity', axis = 1)

Для столбца title

In [ ]:
#отбрасываем финальные пробелы
def remove_space(s):
    return s.rstrip()

df['title'] = df['title'].astype(str).apply(lambda x: remove_space(x))

# Автоматические замены


Базовые замены для всей таблицы

In [ ]:
#заменяем ’ё’ → ’е’
df = df.replace('ё', 'е', regex=True)
#заменяем ’  ’ [два пробела] → ’ ’ [один пробел]
df = df.replace({r's{2,}': ' '}, regex=True)
#заменяем ’ - ’ [пробел-минус-пробел] → ’ — ’ [пробел-длинное тире-пробел]
df = df.replace(" - ", " — ", regex=True)
#заменяем ’ – ’ [пробел-среднее тире-пробел] → ’ — ’ [пробел-длинное тире-пробел]
df = df.replace(" – ", " — ", regex=True)
#заменяем ’...’ [три точки] → ’…’ [многоточие]
df = df.replace("...", "…")
#заменяем ’ ’ [пробел на конце ячейки] — ’’ [отсутствие пробела]
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

Замены для столбца source_id

In [ ]:
#заменяем в "source_id" “-” на значение из source_year
df['source_id'] = np.where(df['source_id'] == '-', df['source_year'], df['source_id'])

Замены для столбца source_year

In [ ]:
#заменяем в "source_year" "1937-38" на "1937"
df['source_year'] = df['source_year'].replace(['1937-38'], 1937)

Замены для столбца author

In [ ]:
#заменяем в "author" "Неизвестный автор." на "Неизвестный автор"
df['author'] = df['author'].replace(['Неизвестный автор.'], 'Неизвестный автор')

#заменяем Фамилия И.О. на Фамилия И. О. — с пробелом между инициалами
def add_space_after_capital_and_dot(s):
    # Use a regular expression to add a space after capital letter and dot (if not last symbol)
    out = re.sub(r'([А-Яа-я])\.(?=[^. ])', r'\1. ', s)
    return out.rstrip()

df['author'] = df['author'].astype(str).apply(lambda x: add_space_after_capital_and_dot(x))

Замены для столбцов type и sort

In [ ]:
#1) type+sort



#заменить желтые ‘фольклор’ на ‘зарубежная’
#остальные ‘фольклор’ — на ‘отечественная’. Весь "фольклор" => в столбец "sort"

df.set_index('uid', inplace= True )
#df.reset_index()
#df
indexes_list = ['3_1715', '3_1716', '3_1717', '3_1718', '3_1719', '3_1722', '3_1733', '3_2485', '3_1778', '3_1318', '3_1783', '3_1784', '3_1321', '3_1786', '3_1787', '3_1801', '3_1802', '3_1322', '3_1814', '3_1820', '3_1825', '3_1832', '3_1833', '3_2488', '3_1845', '3_1847', '3_1853']
#индексы получены через замену в копии (0_сopy_заменить) желтых ячеек на "заменить"
#df_index = df_1[df_1['type'] == 'заменить']
#indexes_list = df_index.reset_index()['uid'].tolist()
#indexes_list

df['sort'] = df['type'].copy()
df['sort'] = df.loc[df['type'] == 'фольклор', 'type']
df.loc[indexes_list, 'type'] ='зарубежная'
df['type'] = df['type'].replace(['фольклор'], 'отечественная')


#в type для значений "духовная": Переместить в sort, а здесь оставить пустую ячейку.

df.loc[df['type'] == 'духовная', 'sort'] = 'духовная'
df['type'] = df['type'].replace(['духовная'], np.NaN)

In [ ]:
#2)  type


#заменяем в "type" "-" на " "
df['type'] = df['type'].replace(['-'], np.NaN)
#заменяем в "type" "древнерусская литература" и "русская" на "отечественная"
df['type'] = df['type'].replace(['древнерусская литература'], 'отечественная')
df['type'] = df['type'].replace(['русская'], 'отечественная')

Замены для столбца oblig_author

In [ ]:
#заменяем в "oblig_author" "нет" на "0", "да" на "1"
df['oblig_author'] = df['oblig_author'].replace(['нет'], 0).replace(['да'], 1)

Замены для столбца oblig_title

In [ ]:
#заменяем в "oblig_title" "нет" на "0", "да" на "1"
df['oblig_title'] = df['oblig_title'].replace(['нет'], 0).replace(['да'], 1)

#заменяем в oblig_title (для source_year кроме ‘2023’) '*' на ‘1’ (для source_year ‘2023’ звездочку оставляем)
asterix2023_indexes = df.index[(df['source_year'] == 2023) & (df['oblig_title'] == '*')].tolist()
df.loc[asterix2023_indexes, 'oblig_title'] ='asterix'
df['oblig_title'] = df['oblig_title'].replace(['*'], 1)
df['oblig_title'] = df['oblig_title'].replace(['asterix'], '*')

Замены для столбца grade

In [ ]:
#заменяем в "grade" "средняя" и " средняя" на "5–9"
df['grade'] = df['grade'].replace(['средняя'], '5–9').replace([' средняя'], '5–9')

#заменяем в "grade" "старшая" на "5–11"
df['grade'] = df['grade'].replace(['старшая'], '5–11')

Замены для столбца profound

In [ ]:
#заменяем в "profound" "да" на "1"
df['profound'] = df['profound'].replace(['да'], 1)

Замены для столбца notes, oblig_title, profound

In [ ]:
#1) notes


#заменяем в "notes" "5,8,9" на "статьи 5, 8, 9"
df['notes'] = df['notes'].replace(['5,8,9'], 'статьи 5, 8, 9')
#заменяем в "notes" "5,8,9 в сокращении" на "статьи 5, 8, 9 (в сокращении)"
df['notes'] = df['notes'].replace(['5,8,9 в сокращении'], 'статьи 5, 8, 9 (в сокращении)')
#заменяем в "notes" "8,9 отрывки" на "статьи 8, 9 (отрывки)"
df['notes'] = df['notes'].replace(['8,9 отрывки'], 'статьи 8, 9 (отрывки)')
#заменяем в "notes" "гл. 2" на "глава 2"
df['notes'] = df['notes'].replace(['гл. 2'], 'глава 2')
#заменяем в "notes" "гл.1, 2, 3" на "главы 1, 2, 3"
df['notes'] = df['notes'].replace(['гл.1, 2, 3'], 'главы 1, 2, 3')
#заменяем в "notes" "первый том" на "том 1"
df['notes'] = df['notes'].replace(['первый том'], 'том 1')
#заменяем в "notes" "статьи 8 и 9" на "статьи 8, 9"
df['notes'] = df['notes'].replace(['статьи 8 и 9'], 'статьи 8, 9')
#заменяем в "notes" "том 1." на "том 1"
df['notes'] = df['notes'].replace(['том 1.'], 'том 1')
#заменяем в "notes" "ч 1." на "часть 1"
df['notes'] = df['notes'].replace(['ч 1.'], 'часть 1')
#заменяем в "notes" "ч.1" на "часть 1"
df['notes'] = df['notes'].replace(['ч.1'], 'часть 1')
#заменяем в "notes" "ч.1, 2" на "части 1, 2"
df['notes'] = df['notes'].replace(['ч.1, 2'], 'части 1, 2')
#заменяем в "notes" "ч.I, II" на "части 1, 2"
df['notes'] = df['notes'].replace(['ч.I, II'], 'части 1, 2')

In [ ]:
#2) notes + oblig_title


#в oblig_title для значений "отрывки": В столбец notes вставить ‘Отрывки по выбору’, а здесь пока оставить.
df.loc[df['oblig_title'] == 'отрывки', 'notes'] = 'Отрывки по выбору'

#в oblig_title для значений "самостоятельно": Скопировать значение в notes, а здесь пока оставить.
df.loc[df['oblig_title'] == 'самостоятельно', 'notes'] = 'самостоятельно'

#в oblig_title для значений "по выбору": Скопировать значение в notes, а здесь заменить на ‘0’
df.loc[df['oblig_title'] == 'по выбору', 'notes'] = 'по выбору'
df['oblig_title'] = df['oblig_title'].replace(['по выбору'], 0)

In [ ]:
#3) notes + profound


#в oblig_title для значений "*в гуманитарных":
#переместить в notes значение ‘Для изучения в гуманитарных классах’ (без звездочки в начале)
#в столбце profound поставить ‘1’, а здесь заменить на ‘0’

df.loc[df['oblig_title'] == '*в гуманитарных', 'notes'] = 'Для изучения в гуманитарных классах'
df.loc[df['oblig_title'] == '*в гуманитарных', 'profound'] = 1
df['oblig_title'] = df['oblig_title'].replace(['*в гуманитарных'], 0)

# Сверки, связки, замены (работа с отдельными файлами)

1) Sorted: уникальные значения в столбцах

In [ ]:
uids = df['uid'].unique()
new_df = pd.DataFrame(data=uids, columns = ['uid'])
#new_df

#uids = df['uid'].unique()
source_ids = df['source_id'].unique()
source_years = df['source_year'].unique()
groups = df['group'].unique()
centuries = df['century'].unique()
authors = df['author'].unique()
titles = df['title'].unique()
genres = df['genre'].unique()
types = df['type'].unique()
sorts = df['sort'].unique()
translateds = df['translated'].unique()
oblig_authors = df['oblig_author'].unique()
oblig_titles = df['oblig_title'].unique()
grades = df['grade'].unique()
sections = df['section'].unique()
numbers_authors = df['number_authors'].unique()
numbers_titles = df['number_titles'].unique()
profounds = df['profound'].unique()
source_issues = df['source_issue'].unique()
title_cycles = df['title_cycle'].unique()
notes = df['notes'].unique()

df_merged = pd.concat([new_df, pd.DataFrame(source_ids), pd.DataFrame(source_years), pd.DataFrame(groups), pd.DataFrame(centuries), pd.DataFrame(authors), pd.DataFrame(titles), pd.DataFrame(genres), pd.DataFrame(types), pd.DataFrame(oblig_authors), pd.DataFrame(oblig_titles), pd.DataFrame(grades), pd.DataFrame(sections), pd.DataFrame(numbers_authors), pd.DataFrame(numbers_titles), pd.DataFrame(profounds), pd.DataFrame(source_issues), pd.DataFrame(title_cycles), pd.DataFrame(translateds), pd.DataFrame(sorts), pd.DataFrame(notes)], axis= 1)
df_merged.columns = ['uid', 'source_id', 'source_year', 'group', 'century', 'author', 'title', 'genre', 'type', 'oblig_author', 'oblig_title', 'grade', 'section', 'number_authors', 'number_titles', 'profound', 'source_issue', 'title_cycle', 'translated', 'sort', 'notes']
df_merged.to_excel('C:/Users/User/Documents/0_ВШЭ_проект/sorted2.xlsx', encoding='utf-8', index = False)
#df_merged

2) Связки: author, title, century, genre, type, sort, translated, notes, title_cycle

In [ ]:
import pandas as pd
import numpy as np

# Путь к файлу
file_path = 'C:/Users/User/Documents/0_ВШЭ_проект/0_replacements.xlsx'

# Чтение всех листов из Excel файла
xlsx = pd.ExcelFile(file_path)
sheets = xlsx.sheet_names

# Обработка каждого листа
processed_sheets = {}
for sheet in sheets:
    # Чтение листа в DataFrame
    df = pd.read_excel(file_path, sheet_name=sheet)

    # Удаление строк с пустыми значениями
    df = df.dropna()

    # Проверка наличия столбца 'uid' перед применением explode
    if 'uid' in df.columns:
        df['uid'] = df['uid'].str.split(', ')
        df = df.explode('uid')

    # Сохранение обработанного DataFrame в словарь
    processed_sheets[sheet] = df


# Запись обработанных данных обратно в новый Excel файл
output_path = 'C:/Users/User/Documents/0_ВШЭ_проект/1_replacements.xlsx'
with pd.ExcelWriter(output_path) as writer:
    for sheet, df_processed in processed_sheets.items():
        df_processed.to_excel(writer, sheet_name=sheet, index=False)

#output_path = 'C:/Users/User/Documents/0_ВШЭ_проект/0_replacements.xlsx'
#with pd.ExcelWriter(output_path) as writer:
 #   for sheet, df_processed in processed_sheets.items():
        # Установка индекса
   #     df_processed = df_processed.set_index()
    #    df_processed.to_excel(writer, sheet_name=sheet)

In [ ]:
main = pd.read_excel('C:/Users/User/Documents/0_ВШЭ_проект/merged2.xlsx', index_col='uid')
#main

sheets_to_update = {
    'author': 'author',
    'title': 'title',
    'type': 'type',
    'genre': 'genre',
    'century': 'century',
    'sort': 'sort',
    'translated': 'translated',
    'notes': 'notes',
    'title_cycle': 'title_cycle'
}

In [ ]:
# Проходим по каждому листу и обновляем соответствующие столбцы
for sheet_name, columns_to_update in sheets_to_update.items():
    replace = pd.read_excel(output_path, sheet_name=sheet_name, index_col='uid')

    # Проходим датасет replace с помощью цикла for и вытаскиваем из него индексы (uid)
    for uid in replace.index:
        if uid in main.index:
            if pd.isna(main.loc[uid, columns_to_update]):
                main.loc[uid, columns_to_update] = replace.loc[uid, columns_to_update]
            else:
                main.loc[uid, columns_to_update] = replace.loc[uid, columns_to_update]

In [ ]:
main.reset_index(inplace=True)
main.to_excel('C:/Users/User/Documents/0_ВШЭ_проект/replacement_result.xlsx', index = False)

3) Замены для программ целиком (со старыми и новыми индексами, по определнным столбцам для старых)

In [ ]:
import pandas as pd
import numpy as np

main = pd.read_excel('C:/Users/User/Documents/0_ВШЭ_проект/try.xlsx', sheet_name='main', index_col='uid')
replace = pd.read_excel('C:/Users/User/Documents/0_ВШЭ_проект/try.xlsx', sheet_name='replace', index_col='uid')

In [ ]:
#Столбцы, которые нужно обновить
columns_to_update = ['source_id', 'source_year', 'group', 'oblig_author', 'oblig_title', 'grade', 'number_authors', 'number_titles', 'source_issue', 'title_cycle', 'notes']

#Проходим датасет replace с помощью цикла for и вытаскиваем из него индексы (uid)
for i in range(len(replace)):
    uid = replace.index[i]

    #Проверяем, есть ли эти индексы в датасете main
    if uid in main.index:
        #Проверяем наличие пустых значений в строке main по этому индексу
        if main.loc[uid][columns_to_update].isnull().any():
            #Заполняем пустые значения с помощью fillna только в нужных столбцах
            main.loc[uid, columns_to_update] = main.loc[uid, columns_to_update].fillna(replace.loc[uid, columns_to_update])
        else:
            #Заменяем данные только в нужных столбцах
            main.loc[uid, columns_to_update] = replace.loc[uid, columns_to_update]
    else:
        #Если индексы не совпали, склеиваем датасеты
        main = pd.concat([main, replace.iloc[[i]]], axis=0)

In [ ]:
main.reset_index(inplace=True)
main.to_excel('C:/Users/User/Documents/0_ВШЭ_проект/results.xlsx', index = False)